In [1]:
import pandas as pd
from transformers import pipeline
import time

/Users/lynn/Documents/Twinkl/grant-applications-app/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/feb-march-data.csv')

print(len(df))
print(df.columns)

df.head(3)

960
Index(['Id', 'Date/Time Requested', 'Giveaway Title', 'Customer Name',
       'Email Address', 'School Name', 'Postal Address', 'Address Line 2',
       'Address City', 'Postcode', 'Additional Info', 'Unnamed: 11'],
      dtype='object')


,Id,Date/Time Requested,Giveaway Title,Customer Name,Email Address,School Name,Postal Address,Address Line 2,Address City,Postcode,Additional Info,Unnamed: 11
0,304399.0,01/03/2025 00:52,March Community Collection,Susan Bushnell,susan.bushnell@googlemail.com,Southfield Junior School,Shrivenham Road,Highworth,Swindon,SN6 7BZ,I would love to use it to spread the love of r...,
1,305004.0,02/03/2025 19:52,March Community Collection,Sarah Arabestani,sarah.a@sandringhamnursery.com,Sandringham Nursery,16 Sandringham Road,Penylan,Cardiff,CF23 5BJ,We would like to introduce early years yoga an...,
2,305493.0,05/03/2025 14:34,March Community Collection,Rebecca Asker,mrsrasker@gmail.com,Newhaven PRU Outreach,Newhaven Gardens,NaN,Greenwich,SE96HR,Â£500 would enable us to set up a small sensor...,


In [3]:
df.columns = df.columns.str.lower().str.replace(' ','_')

df.columns

Index(['id', 'date/time_requested', 'giveaway_title', 'customer_name',
       'email_address', 'school_name', 'postal_address', 'address_line_2',
       'address_city', 'postcode', 'additional_info', 'unnamed:_11'],
      dtype='object')

In [4]:
# Initialize the emotion classification pipeline
emotion_classifier = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions', top_k=None)

Device set to use mps:0


Let's th eclassifier for a single application to know what kind of output we can expect

In [5]:
# first let's find a long applciation

df['word_count'] = df['additional_info'].apply(lambda x: len(str(x).split()))

df[['additional_info', 'word_count']].head(3)

,additional_info,word_count
0,I would love to use it to spread the love of r...,69
1,We would like to introduce early years yoga an...,46
2,Â£500 would enable us to set up a small sensor...,86


In [6]:
test_application = df.loc[df['word_count'].idxmax(), 'additional_info']

print(test_application)

Horton Grange Primary School is located in the Great Horton area of Bradford, West Yorkshire, serving a well-established Pakistani community. The school caters to children aged 2 to 11, with a total enrollment of approximately 726 pupils. The student body is diverse, with 91% of students using English as an additional language. Additionally, 21% of students have Special Educational Needs, and 32 students are considered disadvantaged. In terms of academic performance, 66% of pupils meet the expected standards in reading, writing, and mathematics at Key Stage 2, with 11% achieving a higher standard. The school's most recent Ofsted inspection in September 2024 rated the quality of education as 'Good' and highlighted 'Outstanding' ratings for both behaviour and attitudes, and personal development. The school is part of the Exceed Academies Trust and is led by Headteacher Miss Rebecca Marshall. It offers a range of extracurricular activities, including educational outings and experiences th

In [9]:
emotions = emotion_classifier(test_application)
emotions

[[{'label': 'neutral', 'score': 0.5856776237487793},
  {'label': 'approval', 'score': 0.4809421896934509},
  {'label': 'admiration', 'score': 0.08151720464229584},
  {'label': 'realization', 'score': 0.027009719982743263},
  {'label': 'optimism', 'score': 0.018872186541557312},
  {'label': 'caring', 'score': 0.010474747978150845},
  {'label': 'disapproval', 'score': 0.008761710487306118},
  {'label': 'annoyance', 'score': 0.0053910380229353905},
  {'label': 'disappointment', 'score': 0.004287778399884701},
  {'label': 'relief', 'score': 0.004131054040044546},
  {'label': 'pride', 'score': 0.004071239847689867},
  {'label': 'joy', 'score': 0.00316843600012362},
  {'label': 'gratitude', 'score': 0.002936755074188113},
  {'label': 'desire', 'score': 0.0022684938739985228},
  {'label': 'sadness', 'score': 0.0020141159184277058},
  {'label': 'love', 'score': 0.0017706562066450715},
  {'label': 'confusion', 'score': 0.0015989093808457255},
  {'label': 'excitement', 'score': 0.001488504232838

In [10]:
more_intense_example = """"I run outdoor learning at my school, we are in a very deprived area, in a central area of Milton Keynes in the middle of a housing estate, the school has very little. I currently pay for as much as possible. We have now got an allotment area, but really need some help with filling it!! We are trying hard to run outdoor learning sessions, as we have so many children without gardens who live in converted shipping containers and their faces when they tackle ceratin skills and plant and see things grow is beautiful. please please help us!!!"""

In [11]:
emotions = emotion_classifier(more_intense_example)

emotions

[[{'label': 'admiration', 'score': 0.6447672843933105},
  {'label': 'desire', 'score': 0.3310450613498688},
  {'label': 'optimism', 'score': 0.08578585088253021},
  {'label': 'approval', 'score': 0.08077544718980789},
  {'label': 'neutral', 'score': 0.0645425021648407},
  {'label': 'disappointment', 'score': 0.034120526164770126},
  {'label': 'caring', 'score': 0.026644788682460785},
  {'label': 'sadness', 'score': 0.014421308413147926},
  {'label': 'annoyance', 'score': 0.01335776224732399},
  {'label': 'disapproval', 'score': 0.012340354733169079},
  {'label': 'excitement', 'score': 0.008422383107244968},
  {'label': 'realization', 'score': 0.008331868797540665},
  {'label': 'love', 'score': 0.007035400252789259},
  {'label': 'curiosity', 'score': 0.006367261987179518},
  {'label': 'pride', 'score': 0.005947182886302471},
  {'label': 'joy', 'score': 0.004438790027052164},
  {'label': 'gratitude', 'score': 0.0034672864712774754},
  {'label': 'relief', 'score': 0.0027506654150784016},


In [12]:
def classify_emotions(text):
    """
    Classifies the emotions in a given text and returns a dictionary of emotions and their scores.
    """
    if isinstance(text, str):
        emotions = emotion_classifier(text)
        # The output is a list of lists of dictionaries. We need to process it.
        if emotions and isinstance(emotions[0], list):
            return {item['label']: item['score'] for item in emotions[0]}
        return {}

In [13]:
start_time = time.time()

df['emotion'] = df['additional_info'].map(classify_emotions)

end_time = time.time()

print(f"Processing Time: {end_time-start_time} seconds")

Processing Time: 17.430683851242065 seconds


In [14]:
df['emotion'].value_counts()

emotion
{'desire': 0.6581346988677979, 'love': 0.19491417706012726, 'approval': 0.11420776695013046, 'neutral': 0.10338450223207474, 'admiration': 0.09521238505840302, 'optimism': 0.061544269323349, 'caring': 0.017690906301140785, 'excitement': 0.014791340567171574, 'joy': 0.00833946093916893, 'realization': 0.007762537803500891, 'disappointment': 0.006389504764229059, 'annoyance': 0.00584241421893239, 'disapproval': 0.004605362191796303, 'gratitude': 0.0036359988152980804, 'curiosity': 0.0034046811051666737, 'pride': 0.0032050127629190683, 'sadness': 0.002815956948325038, 'anger': 0.0022237244993448257, 'disgust': 0.0021591379772871733, 'remorse': 0.0015994398854672909, 'confusion': 0.001582510769367218, 'relief': 0.0013734428212046623, 'fear': 0.0010202437406405807, 'surprise': 0.000994302099570632, 'nervousness': 0.0008488827734254301, 'amusement': 0.0007571439491584897, 'grief': 0.0007330116932280362, 'embarrassment': 0.00048045202856883407}                       2
{'neutral': 0.71

In [18]:
def get_top_emotion(emotion_dict):
    """
    Returns the emotion with the highest score from a dictionary of emotions.
    """
    if not emotion_dict:
        return None, 0.0
    top_emotion = max(emotion_dict, key=emotion_dict.get)
    return top_emotion, emotion_dict[top_emotion]

# Apply the function to the 'emotions' column
df[['top_emotion', 'top_emotion_score']] = df['emotion'].apply(get_top_emotion).apply(pd.Series)

In [20]:
df['top_emotion'].value_counts()

top_emotion
neutral           319
desire            212
approval           88
love               79
gratitude          58
admiration         51
optimism           36
disappointment     16
excitement         15
caring             12
sadness            11
joy                 9
realization         2
nervousness         1
confusion           1
disapproval         1
Name: count, dtype: int64

In [23]:
not_list = ['neutral', 'approval', 'love', 'admiration', 'optimism', 'excitement', 'joy']

not_df = df[~df['top_emotion'].isin(not_list)]

In [25]:
len(not_df)

363